##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ejemplos de migración: Estimadores enlatados

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/canned_estimators">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver en TensorFlow.org</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/canned_estimators.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/canned_estimators.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/migrate/canned_estimators.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>

Los estimadores enlatados (o prefabricados) se han usado tradicionalmente en TensorFlow 1 como formas rápidas y sencillas de entrenar modelos para una variedad de casos de uso típicos. TensorFlow 2 proporciona sustitutos aproximados directos para un número de ellos por medio de modelos Keras. Para aquellos estimadores enlatados que no tienen sustitutos incorporados en TensorFlow 2, puede construir su propio reemplazo con bastante facilidad.

Esta guía le mostrará algunos ejemplos de equivalencias directas y sustituciones personalizadas para demostrar cómo los modelos derivados de `tf.estimator` de TensorFlow 1 se pueden migrar a TensorFlow 2 con Keras.

En concreto, esta guía incluye ejemplos de migración:

- De `tf.estimator`'s `LinearEstimator`, `Classifier` o `Regressor` en TensorFlow 1 a Keras `tf.compat.v1.keras.models.LinearModel` en TensorFlow 2
- De `tf.estimator`'s `DNNEstimator`, `Classifier` o `Regressor` en TensorFlow 1 a una DNN personalizada de Keras ModelKeras en TensorFlow 2
- De `tf.estimator`'s `DNNLinearCombinedEstimator`, `Classifier` o `Regressor` en TensorFlow 1 a `tf.compat.v1.keras.models.WideDeepModel` en TensorFlow 2
- De `tf.estimator`'s `BoostedTreesEstimator`, `Classifier` o `Regressor` en TensorFlow 1 a `tfdf.keras.GradientBoostedTreesModel` en TensorFlow 2

Un precursor común para el entrenamiento de un modelo es el preprocesamiento de características, que se realiza para los modelos de Estimador de TensorFlow 1 con `tf.feature_column`. Para obtener más información sobre el preprocesamiento de características en TensorFlow 2, consulte [esta guía sobre la migración de columnas de características a la API de capas de preprocesamiento de Keras](migrating_feature_columns.ipynb).

## Preparación

Comience con un par de importaciones necesarias de TensorFlow,

In [ ]:
!pip install tensorflow_decision_forests

In [ ]:
import keras
import pandas as pd
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import tensorflow_decision_forests as tfdf


prepare algunos datos sencillos para demostrar a partir del conjunto de datos estándar Titanic,

In [ ]:
x_train = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
x_eval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
x_train['sex'].replace(('male', 'female'), (0, 1), inplace=True)
x_eval['sex'].replace(('male', 'female'), (0, 1), inplace=True)

x_train['alone'].replace(('n', 'y'), (0, 1), inplace=True)
x_eval['alone'].replace(('n', 'y'), (0, 1), inplace=True)

x_train['class'].replace(('First', 'Second', 'Third'), (1, 2, 3), inplace=True)
x_eval['class'].replace(('First', 'Second', 'Third'), (1, 2, 3), inplace=True)

x_train.drop(['embark_town', 'deck'], axis=1, inplace=True)
x_eval.drop(['embark_town', 'deck'], axis=1, inplace=True)

y_train = x_train.pop('survived')
y_eval = x_eval.pop('survived')

In [ ]:
# Data setup for TensorFlow 1 with `tf.estimator`
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((dict(x_train), y_train)).batch(32)


def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices((dict(x_eval), y_eval)).batch(32)


FEATURE_NAMES = [
    'age', 'fare', 'sex', 'n_siblings_spouses', 'parch', 'class', 'alone'
]

feature_columns = []
for fn in FEATURE_NAMES:
  feat_col = tf1.feature_column.numeric_column(fn, dtype=tf.float32)
  feature_columns.append(feat_col)

y cree un método para crear instancias de un optimizador de muestra simplista para utilizar con varios modelos del Estimador TensorFlow 1 y Keras de TensorFlow 2.

In [ ]:
def create_sample_optimizer(tf_version):
  if tf_version == 'tf1':
    optimizer = lambda: tf.keras.optimizers.legacy.Ftrl(
        l1_regularization_strength=0.001,
        learning_rate=tf1.train.exponential_decay(
            learning_rate=0.1,
            global_step=tf1.train.get_global_step(),
            decay_steps=10000,
            decay_rate=0.9))
  elif tf_version == 'tf2':
    optimizer = tf.keras.optimizers.legacy.Ftrl(
        l1_regularization_strength=0.001,
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=0.1, decay_steps=10000, decay_rate=0.9))
  return optimizer

## Ejemplo 1: Migración desde LinearEstimator

### TensorFlow 1: Uso del LinearEstimator

En TensorFlow 1, puede utilizar `tf.estimator.LinearEstimator` para crear un modelo lineal de referencia para problemas de regresión y clasificación.

In [ ]:
linear_estimator = tf.estimator.LinearEstimator(
    head=tf.estimator.BinaryClassHead(),
    feature_columns=feature_columns,
    optimizer=create_sample_optimizer('tf1'))

In [ ]:
linear_estimator.train(input_fn=_input_fn, steps=100)
linear_estimator.evaluate(input_fn=_eval_input_fn, steps=10)

### TensorFlow 2: Uso de Keras LinearModel

En TensorFlow 2, puede crear una instancia de Keras `tf.compat.v1.keras.models.LinearModel` que es el sustituto de `tf.estimator.LinearEstimator`. La ruta `tf.compat.v1.keras` se utiliza para indicar que el modelo prefabricado existe por motivos de compatibilidad.

In [ ]:
linear_model = tf.compat.v1.keras.experimental.LinearModel()
linear_model.compile(loss='mse', optimizer=create_sample_optimizer('tf2'), metrics=['accuracy'])
linear_model.fit(x_train, y_train, epochs=10)
linear_model.evaluate(x_eval, y_eval, return_dict=True)

## Ejemplo 2: Migración desde DNNEstimator

### TensorFlow 1: Uso de DNNEstimator

En TensorFlow 1, puede utilizar `tf.estimator.DNNEstimator` para crear un modelo básico de red neuronal profunda (DNN) para problemas de regresión y clasificación.

In [ ]:
dnn_estimator = tf.estimator.DNNEstimator(
    head=tf.estimator.BinaryClassHead(),
    feature_columns=feature_columns,
    hidden_units=[128],
    activation_fn=tf.nn.relu,
    optimizer=create_sample_optimizer('tf1'))

In [ ]:
dnn_estimator.train(input_fn=_input_fn, steps=100)
dnn_estimator.evaluate(input_fn=_eval_input_fn, steps=10)

### TensorFlow 2: Uso de Keras para crear un modelo DNN personalizado

En TensorFlow 2, puede crear un modelo DNN personalizado para sustituir a uno generado por `tf.estimator.DNNEstimator`, con niveles similares de personalización especificados por el usuario (por ejemplo, como en el ejemplo anterior, la capacidad de personalizar un optimizador de modelo elegido).

Se puede utilizar un flujo de trabajo similar para sustituir `tf.estimator.experimental.RNNEstimator` por un modelo de red neuronal recurrente (RNN) de Keras. Keras proporciona una serie de opciones incorporadas y personalizables mediante `tf.keras.layers.RNN`, `tf.keras.layers.LSTM`, y `tf.keras.layers.GRU`. Para obtener más información, consulte la sección *Capas RNN incorporadas: un ejemplo sencillo* de la guía [RNN con Keras](https://www.tensorflow.org/guide/keras/rnn).

In [ ]:
dnn_model = tf.keras.models.Sequential(
    [tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(1)])

dnn_model.compile(loss='mse', optimizer=create_sample_optimizer('tf2'), metrics=['accuracy'])

In [ ]:
dnn_model.fit(x_train, y_train, epochs=10)
dnn_model.evaluate(x_eval, y_eval, return_dict=True)

## Ejemplo 3: Migración desde DNNLinearCombinedEstimator

### TensorFlow 1: Uso de DNNLinearCombinedEstimator

En TensorFlow 1, puede utilizar `tf.estimator.DNNLinearCombinedEstimator` para crear un modelo básico combinado para resolver problemas de regresión y clasificación con capacidad de personalización tanto para sus componentes lineales como DNN.

In [ ]:
optimizer = create_sample_optimizer('tf1')

combined_estimator = tf.estimator.DNNLinearCombinedEstimator(
    head=tf.estimator.BinaryClassHead(),
    # Wide settings
    linear_feature_columns=feature_columns,
    linear_optimizer=optimizer,
    # Deep settings
    dnn_feature_columns=feature_columns,
    dnn_hidden_units=[128],
    dnn_optimizer=optimizer)

In [ ]:
combined_estimator.train(input_fn=_input_fn, steps=100)
combined_estimator.evaluate(input_fn=_eval_input_fn, steps=10)

### TensorFlow 2: Uso de Keras WideDeepModel

En TensorFlow 2, puede crear una instancia de Keras `tf.compat.v1.keras.models.WideDeepModel` para sustituir a una generada por `tf.estimator.DNNLinearCombinedEstimator`, con niveles similares de personalización especificados por el usuario (por ejemplo, como en el ejemplo anterior, la capacidad de personalizar un optimizador del modelo elegido).

Este `WideDeepModel` se construye sobre la base de un `LinearModel` constituyente y un Modelo DNN personalizado, ambos discutidos en los dos ejemplos anteriores. Si lo desea, también puede utilizar un modelo lineal personalizado en vez del modelo `LinearModel` incorporado en Keras.

Si desea construir su propio modelo en vez de utilizar un estimador enlatado, consulte la guía [Modelo secuencial de Keras](https://www.tensorflow.org/guide/keras/sequential_model). Para obtener más información sobre el entrenamiento personalizado y los optimizadores, consulte la guía [Entrenamiento personalizado: recorrido](https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough).

In [ ]:
# Create LinearModel and DNN Model as in Examples 1 and 2
optimizer = create_sample_optimizer('tf2')

linear_model = tf.compat.v1.keras.experimental.LinearModel()
linear_model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
linear_model.fit(x_train, y_train, epochs=10, verbose=0)

dnn_model = tf.keras.models.Sequential(
    [tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(1)])
dnn_model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
combined_model = tf.compat.v1.keras.experimental.WideDeepModel(linear_model,
                                                               dnn_model)
combined_model.compile(
    optimizer=[optimizer, optimizer], loss='mse', metrics=['accuracy'])
combined_model.fit([x_train, x_train], y_train, epochs=10)
combined_model.evaluate(x_eval, y_eval, return_dict=True)

## Ejemplo 4: Migración desde BoostedTreesEstimator

### TensorFlow 1: Uso de BoostedTreesEstimator

En TensorFlow 1, podría utilizar `tf.estimator.BoostedTreesEstimator` para crear una línea de base para crear una línea de base del modelo Gradient Boosting utilizando un conjunto de árboles de decisión para resolver problemas de regresión y clasificación. Esta funcionalidad ya no está incluida en TensorFlow 2.

```
bt_estimator = tf1.estimator.BoostedTreesEstimator(
    head=tf.estimator.BinaryClassHead(),
    n_batches_per_layer=1,
    max_depth=10,
    n_trees=1000,
    feature_columns=feature_columns)
```

```
bt_estimator.train(input_fn=_input_fn, steps=1000)
bt_estimator.evaluate(input_fn=_eval_input_fn, steps=100)
```

### TensorFlow 2: Uso de los Bosques de decisiones de TensorFlow

En TensorFlow 2, `tf.estimator.BoostedTreesEstimator` se sustituye por [tfdf.keras.GradientBoostedTreesModel](https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/GradientBoostedTreesModel#attributes) del paquete [Bosques de decisiones de TensorFlow](https://www.tensorflow.org/decision_forests).

Los Bosques de decisiones de TensorFlow proporcionan varias ventajas sobre el `tf.estimator.BoostedTreesEstimator`, especialmente en cuanto a calidad, velocidad, facilidad de uso y flexibilidad. Para aprender sobre los Bosques de decisiones de TensorFlow, comience con el [colab para principiantes](https://www.tensorflow.org/decision_forests/tutorials/beginner_colab).

El siguiente ejemplo muestra cómo entrenar un modelo de Gradient Boosted Trees usando TensorFlow 2:

Instale los Bosques de decisiones de TensorFlow.

In [ ]:
!pip install tensorflow_decision_forests

Cree un conjunto de datos en TensorFlow. Tenga en cuenta que los bosques de decisiones admiten de forma nativa muchos tipos de características y no necesitan preprocesamiento.

In [ ]:
train_dataframe = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
eval_dataframe = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')

# Convert the Pandas Dataframes into TensorFlow datasets.
train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(train_dataframe, label="survived")
eval_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(eval_dataframe, label="survived")

Entrene el modelo en el conjunto de datos `train_dataset`.

In [ ]:
# Use the default hyper-parameters of the model.
gbt_model = tfdf.keras.GradientBoostedTreesModel()
gbt_model.fit(train_dataset)

Evalúe la calidad del modelo en el conjunto de datos `eval_dataset`.

In [ ]:
gbt_model.compile(metrics=['accuracy'])
gbt_evaluation = gbt_model.evaluate(eval_dataset, return_dict=True)
print(gbt_evaluation)

Gradient Boosted Trees es solo uno de los muchos algoritmos de bosques de decisiones disponibles en los Bosques de decisiones de TensorFlow. Por ejemplo, Random Forests (disponible como [tfdf.keras.GradientBoostedTreesModel](https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/RandomForestModel) es muy resistente al sobreajuste) mientras que CART (disponible como [tfdf.keras.CartModel](https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/CartModel)) es genial para la interpretación de modelos.

En el siguiente ejemplo, entrene y represente gráficamente un modelo de Random Forest.

In [ ]:
# Train a Random Forest model
rf_model = tfdf.keras.RandomForestModel()
rf_model.fit(train_dataset)

# Evaluate the Random Forest model
rf_model.compile(metrics=['accuracy'])
rf_evaluation = rf_model.evaluate(eval_dataset, return_dict=True)
print(rf_evaluation)

En el ejemplo final, entrene y evalúe un modelo CART.

In [ ]:
# Train a CART model
cart_model = tfdf.keras.CartModel()
cart_model.fit(train_dataset)

# Plot the CART model
tfdf.model_plotter.plot_model_in_colab(cart_model, max_depth=2)